In [9]:
""" 
The purpose of this script is to further generalize the script to 
adapt to different experiments and temperature at the same time
"""
import pybamm as pb;import pandas as pd;import numpy as np;
import os, json,openpyxl,traceback,multiprocessing,scipy.optimize,sys
import matplotlib.pyplot as plt;
import imageio,timeit,random,time, signal
from scipy.io import savemat,loadmat;
from pybamm import constants,exp;import matplotlib as mpl

In [10]:
########################     Global settings!!!
rows_per_file = 3;  Scan_end_end = 12;
purpose_i = "Full_Exp23_Paper_8_1"

On_HPC =  False  
Runshort  =   False                    # a long run or a quick test



if On_HPC:
    i_bundle = int(os.environ["PBS_ARRAY_INDEX"])
else:
    i_bundle = 1; # manually specify
Scan_start = (i_bundle-1)*rows_per_file+1;    
Scan_end   = min(Scan_start + rows_per_file-1, Scan_end_end)    
purpose = f"{purpose_i}_Case_{Scan_start}_{Scan_end}"
# interpetation: Simnon suggested, with cracking activation, heat transfer
para_csv = f"Bundle_{i_bundle}.csv"  # name of the random file to get parameters


if On_HPC:                          # Run on HPC
    Path_csv = f"InputData/{purpose_i}/" 
    Path_NiallDMA = "InputData/" 
    BasicPath=os.getcwd() 
    Para_file = Path_csv +  para_csv
else:
    import sys  
    str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
    str_path_1 = os.path.abspath(
        os.path.join(str_path_0,"wip/Rio_Code/Fun_P2"))
    sys.path.append(str_path_1) 
    Path_NiallDMA = os.path.expanduser(
        "~/EnvPBGEM_Linux/SimSave/InputData/") # for Linux
    BasicPath =  os.path.expanduser(
        "~/EnvPBGEM_Linux/SimSave/P2_R9_Dim")
    Para_file = BasicPath+f'/Get_Random_sets2/{purpose_i}/'+para_csv
    
if not os.path.exists(BasicPath +"/"+ purpose):
    os.mkdir(BasicPath +"/"+ purpose);
# import all functions 
from Fun_P2 import * 

# Load the random file
parameter_names, combinations = load_combinations_from_csv(Para_file)


pool_no = len(combinations)
Indexs  = np.arange(Scan_start-1,Scan_end)
index_list = Indexs+1
""" para_dict_Same = {
   "Total ageing cycles":int(tot_cyc), # 
   "Ageing cycles between RPT":int(cyc_age), # 
   "Update cycles for ageing": int(cyc_age),
} """
# Get all para
Para_dict_list = []
# get all dictionaries
for combination in combinations:
    input_dict = {}
    for parameter_name,para_value in zip(parameter_names,combination ):
        input_dict[parameter_name] = para_value
    Para_dict_list.append(input_dict)
print(f"Total scan case is {len(Para_dict_list)}")

Total scan case is 3


In [11]:
# define global index and dict for all experiment data - prepare for read!
Exp_Path = [
    "Expt 1 - Si-based Degradation/",
    "Expt 2,2 - C-based Degradation 2/",
    "Expt 3 - Cathode Degradation and Li-Plating/",
    "Expt 4 - Drive Cycle Aging (Control)/",
    "Expt 5 - Standard Cycle Aging (Control)/",]
Exp_head = [
    "Expt 1",
    "Expt 2,2",
    "Expt 3",
    "Expt 4",
    "Expt 5",]
Exp_1_Cell = ["A","B","J","D","E","F","K","L","M"];
Exp_1_Temp = {
    "A":"10","B":"10","J":"10",
    "D":"25","E":"25","F":"25",
    "K":"40","L":"40","M":"40",}
Temp_Cell_Exp_1 = {
    "10":["A","B","J"],
    "25":["D","E","F"],
    "40":["K","L","M"],}
Exp_2_Cell = ["A","B","C","D","E","F"];
Exp_2_Temp = {
    "A":"10","B":"10",
    "C":"25","D":"25",
    "E":"40","F":"40",}
Temp_Cell_Exp_2 = {
    "10":["A","B"],
    "25":["C","D"],
    "40":["E","F"],}
Exp_3_Cell = ["A","B","C","D","E","F","G","H","I"];
Exp_3_Temp = {
    "A":"10","B":"10","C":"10",
    "D":"25","E":"25","F":"25",
    "G":"40","H":"40","I":"40"}
Temp_Cell_Exp_3 = {
    "10":["A","B","C"],
    "25":["D","E","F"],
    "40":["G","H","I"],}
Exp_4_Cell = ["A","B","C","D","E","F","G","H"];
Exp_4_Temp = {
    "A":"10","B":"10","C":"10",
    "D":"25","E":"25",
    "F":"40","G":"40","H":"40",}
Temp_Cell_Exp_4 = {
    "10":["A","B","C"],
    "25":["D","E",],
    "40":["F","G","H"],}
Exp_5_Cell = ["A","B","C","D","E","F","G","H"];
Exp_5_Temp = {
    "A":"10","B":"10","C":"10",
    "D":"25","E":"25",
    "F":"40","G":"40","H":"40",}
Temp_Cell_Exp_5 = {
    "10":["A","B","C"],
    "25":["D","E",],
    "40":["F","G","H"],}
Exp_All_Cell  = [Exp_1_Cell,Exp_2_Cell,Exp_3_Cell,Exp_4_Cell,Exp_5_Cell]
Exp_Temp_Cell = [Exp_1_Temp,Exp_2_Temp,Exp_3_Temp,Exp_4_Temp,Exp_5_Temp]
Temp_Cell_Exp_All = [Temp_Cell_Exp_1,Temp_Cell_Exp_2,Temp_Cell_Exp_3,Temp_Cell_Exp_4,Temp_Cell_Exp_5]
Mark_Cell_All = [
    {
    "A":"o","B":">","J":"v",
    "D":"o","E":">","F":"v",
    "K":"o","L":">","M":"v",},
    {
    "A":"10","B":"10",
    "C":"25","D":"25",
    "E":"40","F":"40",},
    {
    "A":"o","B":">","C":"v",
    "D":"o","E":">","F":"v",
    "G":"o","H":">","I":"v",},
    {
    "A":"o","B":">","C":"v",
    "D":"o","E":">",
    "F":"o","G":">","H":"v",},
    {
    "A":"o","B":">","C":"v",
    "D":"o","E":">",
    "F":"o","G":">","H":"v",}]
Color_Cell_All = [
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"J":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],"F":[0, 0, 0,0.7],
    "K":[1,0,0,0.4],"L":[1,0,0,0.4],"M":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"C":[0, 0, 0,0.7],
    "E":[1,0,0,0.4],"F":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"C":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],"F":[0, 0, 0,0.7],
    "G":[1,0,0,0.4],"H":[1,0,0,0.4],"I":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"C":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],
    "F":[1,0,0,0.4],"G":[1,0,0,0.4],"H":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"C":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],
    "F":[1,0,0,0.4],"G":[1,0,0,0.4],"H":[1,0,0,0.4],}]
# Update 23-06-17 
#   para="Total heat transfer coefficient [W.m-2.K-1]";
#   output="Volume-averaged cell temperature [C]"
########################  Output  ########################
keys_loc_RPT = [ # MAY WANT TO SELECT AGEING CYCLE later
    # Default output:
    "x [m]",
    "x_n [m]",
    "x_s [m]",
    "x_p [m]",
    # default: end; 
    "CCend Porosity",
    "CCend Negative electrode interfacial current density [A.m-2]",
    "CCend Electrolyte potential [V]",
    "CCend Electrolyte concentration [mol.m-3]",
    "CCend Negative electrode reaction overpotential [V]",
    "CCend Negative particle surface concentration [mol.m-3]",
    #"CCend Negative electrode roughness ratio",
    #"CCend Total SEI on cracks thickness [m]",

    "CDend Porosity",
    "CDend Negative electrode interfacial current density [A.m-2]",
    "CDend Electrolyte potential [V]",
    "CDend Electrolyte concentration [mol.m-3]",
    "CDend Negative electrode reaction overpotential [V]",
    "CDend Negative particle surface concentration [mol.m-3]",
    #"CDend Negative electrode roughness ratio",
    #"CDend Total SEI on cracks thickness [m]",
    #"REend Total SEI on cracks thickness [m]",
]
keys_tim_RPT = [
    # default: CD
    "CD Time [h]",
    "CD Terminal voltage [V]",
    "CD Anode potential [V]",    # self defined
    "CD Cathode potential [V]",  # self defined
    "CC Time [h]",
    "CC Terminal voltage [V]",
    "CC Anode potential [V]",    # self defined
    "CC Cathode potential [V]",  # self defined
]
keys_cyc_RPT = [   # default: CDend
    "Discharge capacity [A.h]",
    "Throughput capacity [A.h]",
    "CDend Total lithium capacity in particles [A.h]",
    "CDend Loss of capacity to lithium plating [A.h]",
    "CDend Loss of capacity to SEI [A.h]",
    "CDend Loss of capacity to SEI on cracks [A.h]",
    #"CDend X-averaged total SEI on cracks thickness [m]",
    #"CDend X-averaged negative electrode roughness ratio",
    "CDend Local ECM resistance [Ohm]",
    "CDsta Negative electrode stoichiometry", 
    "CDend Negative electrode stoichiometry",
    "CDsta Positive electrode stoichiometry", 
    "CDend Positive electrode stoichiometry",
    "CDend Negative electrode capacity [A.h]",
    "CDend Positive electrode capacity [A.h]",
]

keys_loc_AGE = [ # MAY WANT TO SELECT AGEING CYCLE later
    # Default output:
    "x [m]",
    "x_n [m]",
    "x_s [m]",
    "x_p [m]",
    # default: end; 
    "CCend Porosity",
    "CCend Negative electrode interfacial current density [A.m-2]",
    "CCend Electrolyte potential [V]",
    "CCend Electrolyte concentration [mol.m-3]",
    "CCend Negative electrode reaction overpotential [V]",
    "CCend Negative particle surface concentration [mol.m-3]",
    "CCend Negative electrode surface potential difference [V]",
    "CCend SEI film overpotential [V]",
    #"CCend Negative electrode roughness ratio",
    #"CCend Total SEI on cracks thickness [m]",

    "CDend Porosity",
    "CDend Negative electrode interfacial current density [A.m-2]",
    "CDend Electrolyte potential [V]",
    "CDend Electrolyte concentration [mol.m-3]",
    "CDend Negative electrode reaction overpotential [V]",
    "CDend Negative particle surface concentration [mol.m-3]",
    #"CDend Negative electrode roughness ratio",
    #"CDend Total SEI on cracks thickness [m]",
    "CDend Negative electrode surface potential difference [V]",
    "CDend SEI film overpotential [V]",
    "CDend Electrolyte diffusivity [m2.s-1]",
    "CDend Electrolyte conductivity [S.m-1]",
]
keys_tim_AGE = [
    # default: CD
    "CD Time [h]",
    "CD Terminal voltage [V]",
    "CD Anode potential [V]",    # self defined
    "CD Cathode potential [V]",  # self defined
    
    "CC Time [h]",
    "CC Terminal voltage [V]",
    "CC Anode potential [V]",    # self defined
    "CC Cathode potential [V]",  # self defined
]
keys_cyc_AGE = [];
keys_all_RPT = [keys_loc_RPT,keys_tim_RPT,keys_cyc_RPT];
keys_all_AGE = [keys_loc_AGE,keys_tim_AGE,keys_cyc_AGE];
keys_all = [keys_all_RPT,keys_all_AGE];


# Write para - 1st round:
Values_1 = []
head_keys = list(Para_dict_list[0].keys())
head_pre = [
    "Scan No","Exp No.","Y or N",
    "Error Tot%","Error SOH %","Error LLI %",
    "Error LAM NE %","Error LAM PE %",
    "Error Res %","Error ageT %","Punish",
    "Dry out",]

head_pos = [ 
    "Total ageing cycles","Ageing cycles between RPT",
    "Update cycles for ageing",
    "exp_AGE_text", "exp_RPT_text",
   "Cap Loss","LLI to LiP",
   "LLI to SEI","LLI to sei-on-cracks",
   "LAM to Neg","LAM to Pos",
   "Vol_Elely_Tot Final", "Vol_Elely_JR Final","Width Final","Error"]
Values_1 .append([*head_pre,*head_keys,*head_pos])
book_name_xlsx = f'Summary_{purpose}.xlsx';
sheet_name_xlsx = 'Output'
Target  = f'/{purpose}/'
write_excel_xlsx(
    BasicPath + Target + book_name_xlsx, 
    sheet_name_xlsx, Values_1)   
Exp_pack = [
    Exp_All_Cell,Temp_Cell_Exp_All,
    Exp_Path,Exp_head,Exp_Temp_Cell,
    book_name_xlsx,]  
# remaining set-up for experiment and create folders

if not os.path.exists(BasicPath +Target+"Mats"):
    os.mkdir(BasicPath +Target +"Mats");
if not os.path.exists(BasicPath +Target+"Plots"):
    os.mkdir(BasicPath +Target+"Plots");
if not os.path.exists(BasicPath +Target+"Excel"):
    os.mkdir(BasicPath +Target+"Excel");

Successfully create a excel file


In [12]:
Para_dict_list[0]

{'Scan No': 1,
 'Exp No.': 2,
 'Ageing temperature': 10.0,
 'Mesh list': '[5,5,5,100,20]',
 'Inner SEI lithium interstitial diffusivity [m2.s-1]': 6.26e-19,
 'Dead lithium decay constant [s-1]': 1e-06,
 'Lithium plating kinetic rate constant [m.s-1]': 1e-10,
 'Negative electrode LAM constant proportional term [s-1]': 1.1e-08,
 'Positive electrode LAM constant proportional term [s-1]': 2.98e-18,
 'Negative electrode cracking rate': 5.29e-25,
 'Outer SEI partial molar volume [m3.mol-1]': 5.41e-05,
 'SEI growth activation energy [J.mol-1]': 10000.0,
 'Negative cracking growth activation energy [J.mol-1]': 0.0,
 'Negative electrode diffusivity activation energy [J.mol-1]': 34000.0,
 'Positive electrode diffusivity activation energy [J.mol-1]': 12000.0,
 'Contact resistance [Ohm]': 0.01,
 'Total heat transfer coefficient [W.m-2.K-1]': 14,
 'Initial electrolyte excessive amount ratio': 1.0,
 'Cycles within RPT': 1,
 'RPT temperature': 25,
 'Para_Set': 'OKane2023',
 'Model option': "{'SEI': '

In [13]:
fs = 13; dpi = 100;
midc_merge_all = [];Sol_RPT_all = [];Sol_AGE_all = [];
Plot_Exp=True;     Timeout=False;  
Return_Sol=False;   Check_Small_Time=True; R_from_GITT = True

midc_merge,Sol_RPT,Call_1Cyc = Run_P2_Excel (
    Para_dict_list[0],BasicPath, Path_NiallDMA, 
    purpose,    Exp_pack, keys_all,dpi,fs,
    Runshort,   Plot_Exp,Timeout,Return_Sol,
    Check_Small_Time, R_from_GITT
) 


Start Now! Scan 1.
Read Exp-2 - Cell A RPT 0
Read Exp-2 - Cell A RPT 1
Read Exp-2 - Cell A RPT 2
Read Exp-2 - Cell A RPT 3
Read Exp-2 - Cell A RPT 4
Read Exp-2 - Cell A RPT 5
Read Exp-2 - Cell A RPT 6
Read Exp-2 - Cell A RPT 7
Read Exp-2 - Cell A RPT 8
Read Exp-2 - Cell A RPT 9
Read Exp-2 - Cell A RPT 10
Read Exp-2 - Cell A RPT 11
Read Exp-2 - Cell A RPT 12
Exp-2 - Cell A doesn't have RPT 13
Exp-2 - Cell A doesn't have RPT 14
Exp-2 - Cell A doesn't have RPT 15
Read Exp-2 - Cell B RPT 0
Read Exp-2 - Cell B RPT 1
Read Exp-2 - Cell B RPT 2
Read Exp-2 - Cell B RPT 3


Read Exp-2 - Cell B RPT 4
Read Exp-2 - Cell B RPT 5
Read Exp-2 - Cell B RPT 6
Read Exp-2 - Cell B RPT 7
Read Exp-2 - Cell B RPT 8
Read Exp-2 - Cell B RPT 9
Read Exp-2 - Cell B RPT 10
Read Exp-2 - Cell B RPT 11
Read Exp-2 - Cell B RPT 12
Exp-2 - Cell B doesn't have RPT 13
Exp-2 - Cell B doesn't have RPT 14
Exp-2 - Cell B doesn't have RPT 15
Read Exp-2 - Cell C RPT 0
Read Exp-2 - Cell C RPT 1
Read Exp-2 - Cell C RPT 2
Read Exp-2 - Cell C RPT 3
Read Exp-2 - Cell C RPT 4
Read Exp-2 - Cell C RPT 5
Read Exp-2 - Cell C RPT 6
Read Exp-2 - Cell C RPT 7
Read Exp-2 - Cell C RPT 8
Read Exp-2 - Cell C RPT 9
Read Exp-2 - Cell C RPT 10
Read Exp-2 - Cell C RPT 11
Read Exp-2 - Cell C RPT 12
Exp-2 - Cell C doesn't have RPT 13
Exp-2 - Cell C doesn't have RPT 14
Exp-2 - Cell C doesn't have RPT 15
Read Exp-2 - Cell D RPT 0
Read Exp-2 - Cell D RPT 1
Read Exp-2 - Cell D RPT 2
Read Exp-2 - Cell D RPT 3
Read Exp-2 - Cell D RPT 4
Read Exp-2 - Cell D RPT 5
Read Exp-2 - Cell D RPT 6
Read Exp-2 - Cell D RPT 7
Read

At t = 231.613 and h = 2.02222e-14, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 231.621 and h = 8.19941e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 81.6202 and h = 9.06343e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 81.6201 and h = 2.24943e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 81.6206 and h = 7.10608e-12, the corrector convergence failed repeatedly or with |h| = hmin.


Scan 1: Finish break-in cycle within 4 minutes, 29 seconds
Scan 1: Finish post-process for break-in cycle within 31.324 s


At t = 33.0396 and h = 5.08633e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 33.0395 and h = 4.33648e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 33.0395 and h = 5.36344e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 33.0396 and h = 2.8846e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 33.0395 and h = 6.89038e-13, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 410.574 and h = 5.85811e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 118.679 and h = 4.15743e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 118.679 and h = 3.47431e-12, the corrector convergence failed repeatedly or with |h| = hmin.


...

Scan 1: Finish for No.5676 RPT cycles within 5 minutes, 15 seconds
Scan 1: Finish post-process for No.5676 RPT cycles within 15.541 s


At t = 24.6022 and h = 2.79061e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 24.6021 and h = 4.01504e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 24.6022 and h = 3.54292e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 24.6022 and h = 2.94621e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 24.6022 and h = 2.8741e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 390.113 and h = 2.66213e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 98.2169 and h = 3.05012e-15, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 98.2179 and h = 1.97707e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 98.2178 and h = 3.04752e-12, the corrector convergence failed repeatedly or with |h| = hmin.


Scan 1: Finish for No.5676 ageing cycles within 42 minutes, 15 seconds
Scan 1: Finish post-process for No.6192 ageing cycles within 131.236 ms


At t = 499.941 and h = 6.27751e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 241.37 and h = 5.31154e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 241.37 and h = 1.41239e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 91.3702 and h = 1.57692e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 127.545 and h = 6.31731e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 127.541 and h = 3.02177e-14, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 127.545 and h = 8.17488e-12, the corrector convergence failed repeatedly or with |h| = hmin.


Scan 1: Finish for No.6192 RPT cycles within 4 minutes, 59 seconds
Scan 1: Finish post-process for No.6192 RPT cycles within 11.204 s
Scan 1: Getting extra variables within 223.400 us
Scan 1: Finish all plots within 7.608 s
Successfully create a excel file
Scan 1: Successfully save json file!
Scan 1: Successfully save mat file!
Scan 1: Try saving within 95.416 ms
Succeed doing something in 9 hours, 21 minutes, 43 seconds
This is the end of No. 1  scan


In [14]:
midc_merge["Discharge capacity [A.h]"]

[4.868379797783549,
 4.845874119255121,
 4.834537072234205,
 4.8237168540174,
 4.813325865401952,
 4.803298397849961,
 4.793586405843329,
 4.7841510619037715,
 4.774963384577858,
 4.765998394038875,
 4.757235357806855,
 4.748656510556295,
 4.740249599754433]

In [15]:
midc_merge['CDend SOH [%]']

[100.0,
 99.53771728042511,
 99.30484623314001,
 99.08259121881817,
 98.8691528872365,
 98.66318153807109,
 98.46369028204678,
 98.26988157501343,
 98.08116011720736,
 97.89701280513724,
 97.71701377884906,
 97.5407981258619,
 97.3681141703971]

In [16]:

pb.plot_voltage_components(
    Sol_0[0].cycles[0].steps[0])

NameError: name 'Sol_0' is not defined

In [ ]:
# Sol_0[0].cycles[-1]["Negative electrode stoichiometry"].entries
#label = ["Diff_Li_SEI = 1e-18","Diff_Li_SEI = 1e-16"] 
vars = [
    "Negative electrode stoichiometry",
    "Positive electrode stoichiometry",
    "SEI interfacial current density [A.m-2]",
    "Negative particle surface concentration [mol.m-3]",
    
]
quick_plot = pb.QuickPlot(
    Sol_0[0].cycles[-1], 
    vars,variable_limits='tight') #     
quick_plot.dynamic_plot();

In [ ]:
Sol_0[-2].cycles

In [ ]:
vars = [
    "Negative electrode stoichiometry",
    "Positive electrode stoichiometry",
    "SEI interfacial current density [A.m-2]",
    "Negative particle surface concentration [mol.m-3]",
    
]
quick_plot = pb.QuickPlot(
    Sol_0[-1].cycles[0], 
    vars,variable_limits='tight') #     
quick_plot.dynamic_plot();